In [1]:
import torch

In [15]:
bytes = torch.tensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                      [11, 12, 13, 14, 15, 16, 17, 18, 19, 20]])
print(bytes)
print(bytes.shape)
print(bytes.shape[0])
print(bytes[:, :-2])
print()

new_bytes = bytes.new(bytes.shape[0], 2)
print(new_bytes)
print(new_bytes.shape)
print()

print(torch.cat((bytes[:, :-2], new_bytes), dim=1))

tensor([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
        [11, 12, 13, 14, 15, 16, 17, 18, 19, 20]])
torch.Size([2, 10])
2
tensor([[ 1,  2,  3,  4,  5,  6,  7,  8],
        [11, 12, 13, 14, 15, 16, 17, 18]])

tensor([[0, 0],
        [0, 0]])
torch.Size([2, 2])

tensor([[ 1,  2,  3,  4,  5,  6,  7,  8,  0,  0],
        [11, 12, 13, 14, 15, 16, 17, 18,  0,  0]])
